In [1]:
import pandas as pd
import numpy as np
import os
import re
%matplotlib inline

In [2]:
from kaqt.foundation.constants import kaqt_dir
from kaqt.foundation.constants import settles_data_path

In [3]:
kaqt_dir = '/Users/kapilsharma/KAQT'

In [9]:
data_path = os.path.join(kaqt_dir, settles_data_path)

In [10]:
data_path

'/home/ksharma/dev/kaqt/data/settles'

In [11]:
r = re.compile("\\S*-Futures_Settles")
all_files = os.listdir(data_path)
all_files = [os.path.join(data_path, f) for f in all_files if r.match(f)]
all_files[:5]

['/home/ksharma/dev/kaqt/data/settles/bpeterson@rcgdirect.com-Futures_Settles-Auto_2014-N86772739.csv.gz',
 '/home/ksharma/dev/kaqt/data/settles/bpeterson@rcgdirect.com-Futures_Settles-Auto_2014-N82985931.csv.gz',
 '/home/ksharma/dev/kaqt/data/settles/bpeterson@rcgdirect.com-Futures_Settles-Auto_2014-N84421108.csv.gz',
 '/home/ksharma/dev/kaqt/data/settles/bpeterson@rcgdirect.com-Futures_Settles-Auto_2014-N81978231.csv.gz',
 '/home/ksharma/dev/kaqt/data/settles/bpeterson@rcgdirect.com-Futures_Settles-Auto_2014-N83109117.csv.gz']

In [12]:
settles_data = []
for file in all_files:
    data = pd.read_csv(file, compression='gzip')
    settles_data.append(data)

settles_data = pd.concat(settles_data)

In [13]:
settles_data.count()

#RIC       137720
Date[L]    137720
Time[L]         0
Type       137720
Settle     137720
dtype: int64

In [32]:
settles_data = settles_data[['#RIC','Date[L]','Settle']]

In [33]:
settles_data.tail()

,#RIC,Date[L],Settle
1457,YIBZ5,22-JUN-2015,98.165
1458,YTCU5,22-JUN-2015,97.005
1459,YTCZ5,22-JUN-2015,97.005
1460,YTTU5,22-JUN-2015,98.030
1461,YTTZ5,22-JUN-2015,98.030


In [34]:
settles_data.columns = ['RIC', 'Date', 'SettlePrice']

In [35]:
settles_data.tail()

,RIC,Date,SettlePrice
1457,YIBZ5,22-JUN-2015,98.165
1458,YTCU5,22-JUN-2015,97.005
1459,YTCZ5,22-JUN-2015,97.005
1460,YTTU5,22-JUN-2015,98.030
1461,YTTZ5,22-JUN-2015,98.030


In [37]:
settles_data['Date'] = pd.to_datetime(settles_data['Date'], format='%d-%b-%Y')

In [38]:
settles_data.tail()

,RIC,Date,SettlePrice
1457,YIBZ5,2015-06-22,98.165
1458,YTCU5,2015-06-22,97.005
1459,YTCZ5,2015-06-22,97.005
1460,YTTU5,2015-06-22,98.030
1461,YTTZ5,2015-06-22,98.030


In [75]:
settles_data.RIC = settles_data.RIC.str.lstrip('1')

In [4]:
old_settles = pd.read_csv("/home/ksharma/dev/kaqt/data/settles/OldSettles.csv")

In [5]:
old_settles.tail()

,RIC,Date,SettlePrice
1964700,ZWZ5,2014-05-23,724.50
1964701,ZWZ5,2014-05-27,712.75
1964702,ZWZ5,2014-05-28,712.50
1964703,ZWZ5,2014-05-29,707.00
1964704,ZWZ5,2014-05-30,706.75


In [13]:
import datetime

In [6]:
old_settles.Date = pd.to_datetime(old_settles.Date, format='%Y-%m-%d').apply(lambda x : int(x.strftime("%Y%m%d")))

In [7]:
from tables import *

In [8]:
class ReutersSettlePrice(IsDescription):
    ric = StringCol(itemsize=10)
    date = Int32Col()
    price = Float64Col()

In [9]:
store = open_file("settles.h5", mode = "w")

In [10]:
root = store.root

In [11]:
old_group = store.create_group(root, "Old")

In [12]:
table = store.create_table(old_group, "old_settles", ReutersSettlePrice)

In [13]:
row = table.row

In [14]:
old_settles.tail()

,RIC,Date,SettlePrice
1964700,ZWZ5,20140523,724.50
1964701,ZWZ5,20140527,712.75
1964702,ZWZ5,20140528,712.50
1964703,ZWZ5,20140529,707.00
1964704,ZWZ5,20140530,706.75


In [15]:
for i in old_settles.index:
    row['ric'] = old_settles.ix[i, 'RIC']
    row['date'] = old_settles.ix[i, 'Date']
    row['price'] = old_settles.ix[i, 'SettlePrice']
    row.append()
table.flush()

In [16]:
table.flush()

In [17]:
store.flush()

In [18]:
store.close()

In [ ]:
store = pd.HDFStore('settles.h5', complevel=9, complib='bzip2')

In [52]:
data = store.get('Old/old_settles')

In [53]:
data.tail()

,date,speed,symbol
1964700,20140523,724.50,ZWZ5
1964701,20140527,712.75,ZWZ5
1964702,20140528,712.50,ZWZ5
1964703,20140529,707.00,ZWZ5
1964704,20140530,706.75,ZWZ5


In [54]:
store.close()